In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (104 kB/s)
Reading package lis

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Demographics").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/22-big-data/day_1/demographics.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("demographics.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [ ]:
# Print the column names
df.columns

['id',
 'name',
 'age',
 'height_m',
 'weight_kg',
 'children',
 'occupation',
 'academic_degree',
 'salary',
 'location']

In [ ]:
# Print out the first 10 rows
df.show(10)

+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
| id|             name|age|height_m|weight_kg|children|          occupation|academic_degree|salary|            location|
+---+-----------------+---+--------+---------+--------+--------------------+---------------+------+--------------------+
|  1|    Glad Gavrieli| 38|    1.52|       74|       0|Computer Systems ...|       Bachelor|    78|           Louisiana|
|  2|  Henrieta Fittes| 34|    1.72|       39|       4|             Teacher|         Master|    44|            Illinois|
|  3|   Peyton Dulanty| 24|     1.8|       47|       5|Senior Quality En...|            PhD|    44|      North Carolina|
|  4|     Denna Morgen| 48|    1.81|       71|       5|   Account Executive|         Master|    81|          California|
|  5|    Camella Izaks| 34|    1.65|       60|       1|   Director of Sales|            PhD|    76|                Ohio|
|  6|     Shara Esposi| 49|     

In [ ]:
# Select the age, height_meter, and weight_kg columns and use describe to show the summary statistics
df.select("age","height_m","weight_kg").describe().show()

+-------+------------------+-------------------+------------------+
|summary|               age|           height_m|         weight_kg|
+-------+------------------+-------------------+------------------+
|  count|              1000|               1000|              1000|
|   mean|            42.961|  1.750380000000002|            64.308|
| stddev|14.166869067623207|0.14157152997018183|15.840077147667067|
|    min|                18|                1.5|                38|
|    max|                67|                2.0|                90|
+-------+------------------+-------------------+------------------+



In [ ]:
# Print the schema to see the types
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- height_m: string (nullable = true)
 |-- weight_kg: string (nullable = true)
 |-- children: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- academic_degree: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- location: string (nullable = true)



In [ ]:
# Rename the Salary column to `Salary (1k)` and show only this new column
df = df.withColumnRenamed('Salary', 'Salary(1k)').select('Salary(1k)').show()

+----------+
|Salary(1k)|
+----------+
|        78|
|        44|
|        44|
|        81|
|        76|
|        68|
|        85|
|        84|
|        88|
|       116|
|        51|
|        66|
|        90|
|        40|
|        96|
|       116|
|        74|
|       103|
|        46|
|       114|
+----------+
only showing top 20 rows



In [ ]:
# Create a new column called `Salary` where the values are the `Salary (1k)` * 1000
# Show the columns `Salary` and `Salary (1k)`